In [1]:
import pandas as pd

import os

In [2]:
full_path_to_csv = 'C:/Users/badri/Desktop/CustomObjectDetection/csv_folder'


full_path_to_images = \
            'C:/Users/badri/Desktop/CustomObjectDetection/Dataset/train/Camera_Microphone_Glasses'

labels = ['Camera','Microphone','Glasses']


classes = pd.read_csv(full_path_to_csv + '/' + 'class-descriptions-boxable.csv',
                      usecols=[0, 1], header=None)


encrypted_strings = []


for v in labels :
    
    sub_classes = classes.loc[classes[1] == v]

    e = sub_classes.iloc[0][0]

    encrypted_strings.append(e)

annotations = pd.read_csv(full_path_to_csv + '/' + 'train-annotations-bbox.csv',
                          usecols=['ImageID',
                                   'LabelName',
                                   'XMin',
                                   'XMax',
                                   'YMin',
                                   'YMax'])

sub_ann = annotations.loc[annotations['LabelName'].isin(encrypted_strings)].copy()

sub_ann['center x'] = ''
sub_ann['center y'] = ''
sub_ann['width'] = ''
sub_ann['height'] = ''

for i in range(len(encrypted_strings)) :

    sub_ann.loc[sub_ann['LabelName'] == encrypted_strings[i], 'classNumber'] = i

sub_ann['center x'] = (sub_ann['XMax'] + sub_ann['XMin']) / 2
sub_ann['center y'] = (sub_ann['YMax'] + sub_ann['YMin']) / 2

sub_ann['width'] = sub_ann['XMax'] - sub_ann['XMin']
sub_ann['height'] = sub_ann['YMax'] - sub_ann['YMin']

r = sub_ann.loc[:, ['ImageID',
                    'classNumber',
                    'center x',
                    'center y',
                    'width',
                    'height']].copy()

os.chdir(full_path_to_images)

for current_dir, dirs, files in os.walk('.') :
    
    for f in files :
        
        if f.endswith('.jpg') :
            
            image_name = f[:-4]
            
            sub_r = r.loc[r['ImageID'] == image_name]

            resulted_frame = sub_r.loc[:, ['classNumber',
                                           'center x',
                                           'center y',
                                           'width',
                                           'height']].copy()

            path_to_save = full_path_to_images + '/' + image_name + '.txt'

            resulted_frame.to_csv(path_to_save, header=False, index=False, sep=' ')